# Rogue Planet Habitability

:::{admonition} [S. Karthik Yadavalli](https://orcid.org/0000-0002-0840-6940) (Harvard University)
:class: author

*Description*: An investigation into whether a rogue planet could receive habitable levels of energy flux from galactic supernovae.

*Intended Audience*: Intermediate to Advanced Undergraduate

*tags*: `libraries:numpy`, `sampling-positions`, `supernovae`, `habitability`, `planets`

*Requirements*: [requirements.txt](./requirements.txt)

*Last Updated: July 23, 2024*
:::


:::{admonition} Learning Objectives
:class: learningobjective
1. Learn how to uniformly sample coordinate locations from a cylindrical coordinate space.
2. Successfully calculate distances in cylindrical coordinates.
3. Be able to calculate flux from luminosity and use it to determine the habitability of a given coordinate, assuming a uniform distribution of SNe explosions.
:::


## Introduction
How many supernovae need to be continuously going off in a region of space for most of that space to be habitable? Imagine you have a rogue planet: one that has been ejected from its host star and is floating around its host galaxy in interstellar space. Imagine it is far enough away from its host star and from all other stars that the flux it receives from stars is negligible. As it floats around in its galaxy, is it possible that the planet can remain habitable just from the light and heat emitted by supernovae going off in the galaxy? How many supernovae would need to be going off in this galaxy so that such a rogue planet could actually be habitable?


In these exercises, you'll create a region of space shaped like a disk (approximating a spiral galaxy) and randomly sample locations in that space where there are planets and where there are supernovae going off. Then, you'll calculate how much energy each planet receives from all the supernovae going off. Finally, you'll calculate how many of those planets are habitable. 

:::{admonition} On Habitability
:class: note
In this exercise, we only briefly explore planet habitability. We assume a planet will be habitable if the flux it receives is larger than the flux Mars receives from the Sun and smaller than the flux received by Venus from the Sun. However, habitability is far more complex than this, and depends on more factors, including the atmosphere of the planet and how regularly the flux received by the planet changes.
::: 

:::{admonition} On Units 
:class: note 
Note that all luminosities in this notebook will be in `cgs` units. Therefore, luminosity will need to be calculated in $\rm{ergs}/\rm{s}$, and fluxes in $\rm{erg}/\rm{s}/\rm{cm^2}$. We will use $\rm{kpc}$ to represent distances within the galaxy, but make sure to convert those distances to $\rm{cm}$ for the sake of calculating fluxes.
:::

## Part 1: Helper Functions

We'll stary by importing some packages we will need.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math


First, we need a function that will allow us to retrieve "N" random locations within our cylindrical disk space. 

::::{exercise}
:class: dropdown 

A scaffold for this function is provided below --- we provide the radius and height of the disk we want to sample from, with `N` defaulting to 10000 locations.

1. Modify the function below such that it samples randomly radii and heights in the disk, which when paired with the theta's being sampled around will produce many random locations.

```
def get_random_locs(radius, height, theta = (0, 2*np.pi), N = int(10000)):
    '''
    This should return a 2d array, where the shape is (N, 3). As in,
    there should be N rows, with each row showing one random location. Each row should have 3 columns showing the r, theta, and z value
    of that location. These locations should be within the radius, heigh, and theta ranges passed in as input
    '''
    
    rs = ...
    zs = ...
    thetas = np.random.rand(N)* (theta[1]-theta[0]) + theta[0]
    
    coordinates = np.array([rs, zs, thetas]).T
    return np.asarray(coordinates)
```
:::{admonition} 
:class: dropdown tip
Recall that the galaxy is a cylinder, and we constructed it in cylindrical coordinates (r, $\theta$, z). You need to choose `N` combinations of r, $\theta$, and z and return a list of these combinations. How can we randomly choose many combinations of (r, $\theta$, z) without favoring the central, inner parts of the cylinder? (Hint: It's not just randomly choosing from the range of radii!)
:::
::::

We will also need a function to compute the distance between any two points within our disk. 

:::{exercise}
:class: dropdown 

Write a function `calc_distance(coord1,coord2)` which takes in two coordinates in cylindrical coordinates and returns the distance between them. 

*Hint: It might be easier to convert the coordinates to cartesian first to more easily compute the distance*. 
:::

Additionally, we need to be able to compute the flux received at a specific distance from a supernova, given its luminosity. 

:::{exercise}
:class: dropdown 

Write a function `calc_flux(luminosity,distance_kpc)` that takes in a luminosity and distance in kpc, and computes the flux in erg/s/cm$^2$ at that location. 

*Hint: Recall that $F=L/(4\pi D^2)$, and remember to convert kpc to cm along the way*.

:::

Finally, we need a helper function which returns whether a value of flux corresponds to a below-habitable (0), habitable (1) or above-habitable (2) level. 

:::{exercise}
:class: dropdown 

Define a function `habitable(flux)` which given a flux returns 0, 1, or 2 depending on whether the flux is below a minimum of 586200 erg/s/cm$^2$, between that and a maximum of 2601300 erg/s/cm$^2$, or above that maximum, respectively. 
:::

## Part 2: Main Function 

This next cell is the workhorse of our calculation. In this function, we will create the galaxy, generate locations in it for both supernovae and planets, and calculate how many of those planets will be habitable. 


::: {exercise}
:class: dropdown 
1. Define a "supernova density": `n_sne_kpc3`, which defines how many supernovae are going off in the galaxy at any given time. This is the first simplification we will make in this calculation. For there to be enough flux in the galaxy that even rogue planets are habitable, there need to be so many supernovae going off in the galaxy continuously that we will not even consider how the light emitted by a single supernova changes over time. We will simply assume that at any point in time, a constant number of supernovae are currently active, and that each one is emitting an equal amount of total luminosity. Therefore, we will define a "supernova density"-how many are going off at a given time per cubic kiloparsec.
2. Define the dimensions of the galaxy: `r_kpc` and `h_kpc`. We will assume the galaxy is a cylinder, with a radius and height in kiloparsecs
3. Using the dimensions of the galaxy, calculate the volume and total number of supernovae going off continuously by multplying the supernova density by the volume of the galaxy
4. Define a pixel count: `n_pixels`. We need to decide how many pixels we want to use to describe our galaxy. How many different coordinate locations do we want to generate, from which we will randomly pick planet locations and supernova locations? Then, we need to generate `n_pixels` number of random coordinate locations in the galaxy: `galaxy_locs`. We will do this in our first helper function: `get_random_locs()`. 
5. Define the number of planets: `n_planets`. Then, generate `n_planets` number of random coordinates: `planet_coordinates` by randomly choosing coordinates from `galaxy_locs`
6. Generate `n_sne` number of random coordinates: sne_coordinates by randomly choosing coordinates from galaxy_locs. Note that I have set the `sne_lum` to $10^{44}~\frac{erg}{s}$. This is the peak luminosity of some of the brightest supernovae we've seen! 
7. Define an array that can keep track of how much flux is absorbed by each planet: `planet_fluxes`. Also, keep track of the number of planets that are habitable: `num_habitable`, the number of planets that don't get enough flux to be habitable: `num_cold`, and the number of panets that get too much flux to be habitable: `num_hot`.
8. Looping through the planets, for each planet, add up the amount of flux it received from each supernova (a double nested for loop, with one looping through the planets, and a nested one looping through the supernovae). Then, append the flux to the `planet_fluxes` list. Then calculate whether that planet, with the flux you just calculated, would be habitable, too hot, or too cold.
9. Finally, return the variable `planet_fluxes` so we can make a histogram with how much flux each planet receives. 

A scaffold for this function is provided below.

```
def create_galaxy():
    #step 1
    n_sne_kpc3 = ... #The number of supernovae going off per cubic kiloparsec
    
    #step 2
    r_kpc = ...
    h_kpc = ...

    #step 3
    vol = np.pi*r_kpc**2*h_kpc
    n_sne = int(n_sne_kpc3 * vol) #The total number of supernovae going off in the galaxy (multiply n_sne_kpc3 by the volume of the galaxy)
    print("At "+str(n_sne_kpc3)+" SNe per cubic kpc, in this galaxy (Volume = "+str(round(vol, 3))+" kpc^3), we get "+str(n_sne)+" SNe going off continuously")
    
    
    #step 4 | Use the get_random_locs() function from the previous cell 
    n_pixels = int(1.0E8) #Number of locations to generate throughout the galaxy. 
    galaxy_locs = ...

    #step 5 randomly sample from those locations for N_planets
    n_planets = 100 #The number of planets evenly distributed throughout the galaxy
    planet_coordinates = ...
    
    
    #step 6 random locations for where supernovae could happen
    sne_coordinates = ...
    sn_lum = 1.0E44 

    #step 7 keeping track of how many planets are habitable, and how many are too cold (not enough flux) or too hot (too much flux) to be habitable
    planet_fluxes = []
    num_habitable = 0
    num_cold = 0
    num_hot = 0

    #step 8
    for planet in range(n_planets):
        flux_planet = 0.0
        for i in range(len(sne_coordinates)):
            dist_kpc = ....
            flux_sn_planet = ...
            flux_planet += flux_sn_planet
        planet_fluxes.append(flux_planet)
        habitability = habitable(flux_planet)

        
        if habitability == 0:
            num_cold+= 1
        elif habitability == 1:
            num_habitable+= 1
        elif habitability == 2:
            num_hot += 1
    num_uninhabitable = num_hot + num_cold
        
    

    print("habitable: "+str(num_habitable))
    print("too cold: "+str(num_cold))
    print("too hot: "+str(num_hot))

    return np.asarray(planet_fluxes)
```
:::

## Part 3: Fraction of Habitable Planets 

Finally we can investigate whether a typical rogue planet could survive in our hypothetical galaxy. 

:::{exercise}
:class: dropdown 

Using your `create_galaxy()` function, compute the fluxes at the set of sampled planet locations, and create a histogram of the fluxes, comparing it to the minimum and maximum allowable fluxes for habitability. Are most the planets in habitable regions?
:::